# Basic example for the usage of reverse feature selection

This is a basic example of how to use the reverse feature selection algorithm. The example generates a synthetic dataset
with 100 irrelevant features and two relevant features. The relevant features have an increased effect size. The
algorithm selects the relevant features and prints the names of the selected features.

### Generate synthetic dataset

In [1]:
# Import required libraries to generate an example dataset
import numpy as np
import pandas as pd

# Number of relevant features to insert
n_relevant_features = 2

# Number of irrelevant features
n_irrelevant_features = 200

# Number of total samples (must be an even number for binary classification)
n_samples = 30

# Set up a random number generator
rng = np.random.default_rng(seed=42)

# Create DataFrame with irrelevant features
data_df = pd.DataFrame({f"random_feature{i+1}": rng.random(n_samples) for i in range(n_irrelevant_features)})

# Insert relevant features with increased effect size
for i in range(n_relevant_features):
    regulated_class = rng.random(n_samples // 2) + (i + 1) * 2
    unregulated_class = rng.random(n_samples // 2) + (i + 1)
    # Concatenate the two classes to form a single relevant feature
    data_df.insert(i, f"relevant_feature{i+1}", np.concatenate((regulated_class, unregulated_class)))

### Insert labels

In [2]:
# Construct binary class labels (15 samples of class 0 and 15 of class 1)
label = np.concatenate((np.zeros(n_samples // 2), np.ones(n_samples // 2)))

# Insert label column at the beginning of the DataFrame
data_df.insert(0, "label", label)

data_df.head()

,label,relevant_feature1,relevant_feature2,random_feature1,random_feature2,random_feature3,random_feature4,random_feature5,random_feature6,random_feature7,...,random_feature191,random_feature192,random_feature193,random_feature194,random_feature195,random_feature196,random_feature197,random_feature198,random_feature199,random_feature200
0,0.0,2.455476,4.368622,0.773956,0.744762,0.668403,0.152312,0.584098,0.826292,0.890792,...,0.823674,0.275511,0.080019,0.819990,0.695934,0.994861,0.311558,0.961172,0.692742,0.385556
1,0.0,2.118858,4.455985,0.438878,0.967510,0.471096,0.696320,0.649847,0.896161,0.893447,...,0.097213,0.886400,0.910893,0.690705,0.845520,0.863600,0.223499,0.104798,0.594323,0.046805
2,0.0,2.569173,4.124443,0.858598,0.325825,0.565236,0.446156,0.084444,0.140249,0.518858,...,0.063284,0.680823,0.744646,0.132451,0.720918,0.023697,0.785206,0.529011,0.147729,0.535446
3,0.0,2.156429,4.020634,0.697368,0.370460,0.764999,0.381021,0.415807,0.554036,0.315929,...,0.758601,0.895672,0.931824,0.114184,0.173988,0.251264,0.959814,0.272569,0.816116,0.995409
4,0.0,2.060958,4.994090,0.094177,0.469556,0.634718,0.301512,0.041614,0.108576,0.772012,...,0.536311,0.155901,0.469990,0.771922,0.366217,0.043342,0.492500,0.264141,0.037275,0.968992


### Set training indices (simulate cross-validation)

In [3]:
# Simulate leave-one-out cross-validation by selecting 29 out of 30 samples for training
train_indices = rng.choice(data_df.index, size=29, replace=False)

### Define meta information

Meta data can be left at its default values. But "random_seeds" must be defined for reproducibility and “train_correlation_threshold” should be tuned if the results are not satisfactory. With this parameter the size of the feature subset selction can be adjusted.

In [4]:
# Generate a diverse list of integer random seeds to initialize the random forests for reproducibility
seeds = [29, 10, 17, 42, 213, 34, 1, 5, 19, 3, 23, 9, 7, 123, 234, 345, 456, 567, 678, 789, 890, 15, 333, 37, 45, 56]

# Meta configuration for the feature selection
meta_data = {
    "n_cpus": 4,
    "random_seeds": seeds,
    # train correlation threshold defines the features correlated to the target to be removed from the training data
    "train_correlation_threshold": 0.7,
}

### Run reverse feature selection

In [5]:
# Import the reverse feature selection function
from reverse_feature_selection.reverse_random_forests import select_feature_subset

# Run the reverse feature selection algorithm (could take a Minute or two)
result_df = select_feature_subset(data_df, train_indices, meta_data=meta_data)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   26.8s
[Parallel(n_jobs=4)]: Done 202 out of 202 | elapsed:  1.5min finished


### Display selected features

In [6]:
# Selected features are those with a score greater than 0, stored in the 'feature_subset_selection' column
print("Selected features:")
result_df[result_df["feature_subset_selection"] > 0]["feature_subset_selection"]

Selected features:


relevant_feature2    0.391421
Name: feature_subset_selection, dtype: float64